In [15]:
# Import des librairies
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Chargement du fichier CSV
df = pd.read_csv("merged_data.csv")

# Afficher les premières lignes pour vérifier
df.head()


,_id_x,genre,age,nationalite,niveauEtudes,sessionId,__v_x,_id_y,textId,associationType,personAType,valueOneA,valueTwoA,forceA,personBType,valueOneB,valueTwoB,forceB,createdAt,__v_y
0,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05bb9dacaf24880d98feb,67a243ea6d8e29a001947946,risk-reward,Robot,8,5,8,Personne âgée,2,5,3,2025-03-23T19:06:33.515Z,0
1,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c02dacaf24880d98fed,67a243ea6d8e29a001947945,risk-reward,Femme petite taille,4,7,5,Robot,6,3,8,2025-03-23T19:07:46.725Z,0
2,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c25dacaf24880d98fef,67a51efa18a0dfe14aac016b,risk-effort,Enfant,5,3,3,Femme petite taille,5,7,5,2025-03-23T19:08:21.185Z,0
3,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c33dacaf24880d98ff1,67a243ea6d8e29a001947947,effort-reward,Enfant,3,3,3,Femme petite taille,7,7,5,2025-03-23T19:08:35.846Z,0
4,67e05b30dacaf24880d98fe8,homme,23,HRV,bac+4,420008aa-5136-48fb-9a6f-4347673fce87,0,67e05c4bdacaf24880d98ff3,67a243ea6d8e29a001947949,effort-reward,Femme grande taille,5,5,7,Homme petite taille,5,5,6,2025-03-23T19:08:59.087Z,0


In [16]:

# Binariser les cibles effort / risque / reward
def binariser_targets(row):
    row["effort"] = 1 if "effort" in row["associationType"] else 0
    row["risque"] = 1 if "risk" in row["associationType"] else 0
    row["reward"] = 1 if "reward" in row["associationType"] else 0
    return row

df_bin = df.copy()
df_bin = df_bin.apply(binariser_targets, axis=1)

# Séparer les features (X) et les cibles (y)
# Variables explicatives (features) → ce sont les colonnes qu'on donne au modèle pour qu’il apprenne
X = df_bin[["genre", "age", "nationalite", "niveauEtudes", "personAType", "personBType"]]
# Variables cibles (targets) → ce sont les valeurs à prédire, ici binarisées :
y = df_bin[["effort", "risque", "reward"]].astype(int)

#  Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prétraitement : encodage + normalisation, dans notre dataset les colonne suivantes sont catégorielles :
#genre, nationalite, niveauEtude, PersonTypeA...B   -> ces colonnes sont encodées avec oneHotEncoder
#cela crrer une nouvelle colonne binaire 0 ou 1 pour chaque catégorie
preprocessor = ColumnTransformer(transformers=[
    ("num", StandardScaler(), ["age"]),
    ("cat", OneHotEncoder(handle_unknown="ignore"), ["genre", "nationalite", "niveauEtudes", "personAType", "personBType"])
])
#------------------------------------------------------------------------------------------
# dans cette partie on met le modèle à tester (RandomForest ici, mais facilement remplaçable)
#exemple de model:-----mettre ça --
#from sklearn.svm import SVC
#base_model = SVC()
#  a la place de ca 
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
multioutput_model = MultiOutputClassifier(base_model)
#---------------------------------------------------------------------------------------------------

#  Construire le pipeline complet (prétraitement + modèle)
pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", multioutput_model)
])

#  Entraîner le pipeline sur les données
pipeline.fit(X_train, y_train)

#  Évaluer le modèle
y_pred = pipeline.predict(X_test)
for i, col in enumerate(["effort", "risque", "reward"]):
    print(f"\n📊 Évaluation du modèle pour la cible : {col.upper()}")
    
    y_true = y_test[col]
    y_hat = y_pred[:, i]
    
    acc = accuracy_score(y_true, y_hat)
    prec = precision_score(y_true, y_hat, zero_division=0)
    rec = recall_score(y_true, y_hat, zero_division=0)
    f1 = f1_score(y_true, y_hat, zero_division=0)
    cm = confusion_matrix(y_true, y_hat)
    
    print(f"🔹 Accuracy     : {acc:.2f}")
    print(f"🔹 Précision    : {prec:.2f}")
    print(f"🔹 Rappel       : {rec:.2f}")
    print(f"🔹 F1-score     : {f1:.2f}")
    print(f"🔹 Matrice de confusion :\n{cm}")


📊 Évaluation du modèle pour la cible : EFFORT
🔹 Accuracy     : 0.65
🔹 Précision    : 0.68
🔹 Rappel       : 0.84
🔹 F1-score     : 0.75
🔹 Matrice de confusion :
[[ 29  64]
 [ 25 134]]

📊 Évaluation du modèle pour la cible : RISQUE
🔹 Accuracy     : 0.66
🔹 Précision    : 0.72
🔹 Rappel       : 0.82
🔹 F1-score     : 0.77
🔹 Matrice de confusion :
[[ 27  54]
 [ 31 140]]

📊 Évaluation du modèle pour la cible : REWARD
🔹 Accuracy     : 0.62
🔹 Précision    : 0.71
🔹 Rappel       : 0.76
🔹 F1-score     : 0.73
🔹 Matrice de confusion :
[[ 23  55]
 [ 41 133]]
